In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy

# linear

In [18]:
M = 0.5
m = 0.2
F0 = 0.1
F1 = 0.1
I = 0.006
g = 9.8
l = 0.3
dt = 0.01

# M,m,F0,F1,I,g,l,dt = sympy.symbols("M m F0 F1 J g l dt")
e1,e2,e3,e4 = sympy.symbols("e1 e2 e3 e4")


p = I*(M+m)+M*m*l**2 # denominator for the A and B matrices
A = (M+m)*m*g*l/p
B = -F1*(M+m)/p
C = F0*m*l/p
D = -m*l/p
E = -m*m*g*l*l/p
F = F1*m*l/p
G = -F0*(I+m*l*l)/p
H = (I+m*l*l)/p

# p,A,B,C,D,E,F,G,H,dt = sympy.symbols("p A B C D E F G H dt")


kp1 = 196.1
kd1 = 34.1
kp2 = 80
kd2 = 85.18

# kp1,kd1,kp2,kd2 = sympy.symbols("kp1 kd1 kp2 kd2")
x_d_r = ( kp1*e1+kd1*e2+A*e1+B*e2-E*D/H*e1-F*D/H*e2-G*D/H*e4-D/H*kd2*e4-D/H*kp2*e3) / C
# u = E/H*e1 +F/H*e2+G/H*e4 +(kp2*dt)*x_d_r/H +kd2/H*e4
u = E/H*e1 +F/H*e2+G/H*e4 +kp2*e3/H +kd2/H*e4

e2_d = A*e1+B*e2+C*e4-D*u
e4_d = -kp2*e3 -kd2*e4
x_d_r_d = ( kp1*e2+kd1*e2_d+A*e2+B*e2_d-E*D/H*e2-F*D/H*e2_d-G*D/H*e4_d-D/H*kd2*e4_d-D/H*kp2*e4) / (C + D/H*kp2*dt)
e3_d = x_d_r_d * dt + e4


j21 = sympy.diff(e2_d,e1)
j22 = sympy.diff(e2_d,e2)
j23 = sympy.diff(e2_d,e3)
j24 = sympy.diff(e2_d,e4)

j31 = sympy.diff(e3_d,e1)
j32 = sympy.diff(e3_d,e2)
j33 = sympy.diff(e3_d,e3)
j34 = sympy.diff(e3_d,e4)

JM = sympy.Matrix([[0, 1, 0, 0],
              [j21,j22,j23,j24],
              [j31,j32,j33,j34],
              [0, 0, -kp2, -kd2]])
JM

Matrix([
[               0,                  1,                0,                0],
[            24.5,  -4.16666666666667,            200.0,           212.95],
[-4.7453137254902, -0.620385620915033, 71.2603921568628, 75.5803849019608],
[               0,                  0,              -80,           -85.18]])

# controllability matrix

In [3]:
Y = sympy.Matrix([[0, 1, 0, 0],
              [A, B, 0, C],
              [0, 0, 0, 1],
              [E, F, 0, G]])
Z = sympy.Matrix([[0],
              [D],
              [0],
              [H]])
controllability = sympy.Matrix([Z,Y*Z,Y*Y*Z,Y*Y*Y*Z]).reshape(4,4)

In [4]:
controllability.rank() # full-rank means controllable

4

In [5]:
controllability.eigenvects()

[(0.0542678119351765 - 5.60854288125143*I,
  1,
  [Matrix([
   [  -0.0812750219100699 - 0.222862759712358*I],
   [-0.0857098719840708 - 0.00601246737240708*I],
   [  -0.240882068347694 + 0.0913942582267512*I],
   [   -0.904164959106837 + 0.229025925743201*I]])]),
 (-7.86459801387932 + 2.58322255833338e-63*I,
  1,
  [Matrix([
   [   0.175423648884227 - 0.145124357659937*I],
   [-0.0569399037152105 + 0.0471052050532939*I],
   [    0.27774187032532 - 0.229770106725077*I],
   [  -0.901149823619323 + 0.745502616893052*I]])]),
 (0.0542678119351765 + 5.60854288125143*I,
  1,
  [Matrix([
   [   0.135426636475377 + 0.231166177230037*I],
   [-0.0564708679236207 + 0.0789138114605124*I],
   [  -0.252944206617124 + 0.143822292179763*I],
   [  -0.850213030556757 + 0.621934567963404*I]])]),
 (-165.708257745987,
  1,
  [Matrix([
   [0.0117502004883562 + 9.1351134127957e-65*I],
   [-0.014943646329836 - 1.4231094001756e-65*I],
   [0.135344255511112 + 3.01859749199677e-64*I],
   [  -1.10826700392494 - 1.

## eigen value calculation

In [7]:
JM.eigenvects()

[(-3.9665179031447 - 4.77604200000583*I,
  1,
  [Matrix([
   [ 0.124319680756402 - 0.0268652141741582*I],
   [  -0.621425630668435 - 0.48719466372628*I],
   [   0.0272252408291383 - 0.4271167764268*I],
   [-0.0513835176900403 + 0.417713061916131*I]])]),
 (-8.36536276799311 - 8.70352878371236e-64*I,
  1,
  [Matrix([
   [  0.0445359107965033 - 0.137472102314831*I],
   [   -0.372559050015731 + 1.15000400634223*I],
   [-0.0217226054199875 + 0.0670526813403734*I],
   [ 0.0226234022084908 - 0.0698332336196301*I]])]),
 (-1.78787593552139 + 1.20572403176722e-63*I,
  1,
  [Matrix([
   [0.0380985757264444 - 0.118673379808608*I],
   [ -0.0681155267189495 + 0.2121732799468*I],
   [ 0.255475527591875 - 0.795781567673434*I],
   [-0.245083602757826 + 0.763411726563662*I]])]),
 (-3.9665179031447 + 4.77604200000583*I,
  1,
  [Matrix([
   [-0.0618857602062137 - 0.141954735582987*I],
   [  0.923452755051736 + 0.267497010178925*I],
   [  0.402683239991128 - 0.330722935464677*I],
   [ -0.376206602026623 + 

In [8]:
(JM+JM.T).eigenvals()

{313.978512935583: 1,
 14.5746262318222: 1,
 -15.5077031195116: 1,
 -349.217985067501: 1}

# nonlinear 

In [16]:
import sympy
e1, e2, e3, e4 = sympy.symbols("e1 e2 e3 e4")
# M,m,F0,F1,J,g,l,dt = sympy.symbols("M m F0 F1 J g l dt")

M = .5
m = 0.2
F0 = 0.1
F1 = 0.1
J = 0.006
g = 9.8
l = 0.3
dt = 0.01

kp1 = 196.10
kd1 = 34.10
kp2 = 80.00
kd2 = 85.18

p = (M+m)*(J+m*l**2) - m**2*l**2*sympy.cos(e1)**2
p_d = 2*m*m*l*l*sympy.cos(e1)*sympy.sin(e1)*e2

A = p*(kp1*e1+kd1*e2) - F1*(M+m)*e2 + (M+m)*m*g*l*sympy.sin(e1)
A += m*l*sympy.cos(e1)/(J+m*l*l) * (F1*m*l*e2*sympy.cos(e1) -F0*(J+m*l*l)*e4 -m*m*g*l*l*sympy.sin(e1)*sympy.cos(e1))
A += m*l*sympy.cos(e1)*(p/(J+m*l*l)*(kd2*e4+kp2*e3))
B = F0*m*l*sympy.cos(e1) # -p*m*l*sympy.cos(e1)*kp2*dt/(J+m*l*l)

u = 1/(J+m*l*l) * (F1*m*l*e2*sympy.cos(e1) +(J+m*l*l)*m*l*e2*e2*sympy.sin(e1)\
                   -F0*(J+m*l*l)*e4 \
                   -m*m*g*l*l*sympy.sin(e1)*sympy.cos(e1)\
                   +p*kp2*e3 \
                   +p*kd2*e4 )

e1_d = e2
e2_d = -F1/p*(M+m)*e2 -m*m*l*l/p*e2*e2*sympy.sin(e1)*sympy.cos(e1)+F0*m*l/p*sympy.cos(e1)*e4 +(M+m)*m*g*l/p*sympy.sin(e1) +m*l/p*sympy.cos(e1)*u
e4_d = -kp2*e3-kd2*e4

A_d = p_d*kp1*e1+p*kp1*e2 +p_d*kd1*e2+p*kd1*e2_d -F1*(M+m)*e2_d+(M+m)*m*g*l*sympy.cos(e1)*e2
A_d += m*l/(J+m*l*l)*( F1*m*l*(e2_d*sympy.cos(e1)*sympy.cos(e1)-e2*2*sympy.cos(e1)*sympy.sin(e1)*e2)\
                      - F0*(J+m*l*l)*(e4_d*sympy.cos(e1)-e4*sympy.sin(e1)*e2)\
                      -m*m*g*l*l*( -sympy.sin(e1)*e2*sympy.sin(2*e1)/2 +sympy.cos(e1)*sympy.cos(2*e1)*e2))

A_d += m*l*kd2/(J+m*l*l)*((p_d*e4+p*e4_d)*sympy.cos(e1)+p*e4*(-sympy.sin(e1))*e2)
A_d += m*l*kp2/(J+m*l*l)*((p_d*e3+p*e4)*sympy.cos(e1)+p*e3*(-sympy.sin(e1))*e2)

B_d = -F0*m*l*sympy.sin(e1)*e2 # - m*l*kp2*dt/(J+m*l*l)*(p_d*sympy.cos(e1)-p*sympy.sin(e1)*e2)

p3 = p*m*l*sympy.cos(e1)*kp2*dt/(J+m*l*l)
x3_d = (A_d*B-B_d*A)/(B*B-p3) * dt + e4

#x_d_r = p*(kp1*e1+kd1*e2) - F1*(M+m)*e2 + m*m*l*l*e2*sympy.sin(e1)*sympy.cos(e1) - (M+m)*m*g*l*sympy.sin(e1) + F0*m*l*sympy.cos(e1)*e4
#x_d_r += m*l*sympy.cos(e1)/(J+m*l*l) * (F1*m*l*e2*sympy.cos(e1) +(J+m*l*l)*m*l*e2**2*sympy.sin(e1) -F0*(J+m*l*l)*e4 -m*m*g*l*l*sympy.sin(e1)*sympy.cos(e1))
#x_d_r += m*l*sympy.cos(e1)*(p/(J+m*l*l)*kd2*e4)
#x_d_r /= (F0*m*l*sympy.cos(e1)-p*m*l*sympy.cos(e1)*kp2*dt/(J+m*l*l))

j21 = sympy.diff(e2_d, e1)
j22 = sympy.diff(e2_d, e2)
j23 = sympy.diff(e2_d, e3)
j24 = sympy.diff(e2_d, e4)

j31 = sympy.diff(x3_d, e1)
j32 = sympy.diff(x3_d, e2)
j33 = sympy.diff(x3_d, e3)
j34 = sympy.diff(x3_d, e4)

JM = sympy.Matrix([[0,1,0,0],
                   [j21,j22,j23,j24],
                   [j31,j32,j33,j34],
                   [0,0,-kp2,-kd2]])

In [17]:
JM.evalf(subs={'e1':0, 'e2':0, 'e3':0, 'e4':0})

Matrix([
[                  0,                  1.0,                 0,                0],
[               24.5,    -4.16666666666667,             200.0,           212.95],
[-0.0220310423304506, -0.00288026096191777, 0.330840236686391, 1.34625394902139],
[                  0,                    0,             -80.0,           -85.18]])

In [19]:
JM.evalf(subs={'e1':0, 'e2':0, 'e3':0, 'e4':0}).eigenvects()

[(-3.9665179031447 - 4.77604200000583*I,
  1,
  [Matrix([
   [ 0.124319680756402 - 0.0268652141741582*I],
   [  -0.621425630668435 - 0.48719466372628*I],
   [   0.0272252408291383 - 0.4271167764268*I],
   [-0.0513835176900403 + 0.417713061916131*I]])]),
 (-8.36536276799311 - 8.70352878371236e-64*I,
  1,
  [Matrix([
   [  0.0445359107965033 - 0.137472102314831*I],
   [   -0.372559050015731 + 1.15000400634223*I],
   [-0.0217226054199875 + 0.0670526813403734*I],
   [ 0.0226234022084908 - 0.0698332336196301*I]])]),
 (-1.78787593552139 + 1.20572403176722e-63*I,
  1,
  [Matrix([
   [0.0380985757264444 - 0.118673379808608*I],
   [ -0.0681155267189495 + 0.2121732799468*I],
   [ 0.255475527591875 - 0.795781567673434*I],
   [-0.245083602757826 + 0.763411726563662*I]])]),
 (-3.9665179031447 + 4.77604200000583*I,
  1,
  [Matrix([
   [-0.0618857602062137 - 0.141954735582987*I],
   [  0.923452755051736 + 0.267497010178925*I],
   [  0.402683239991128 - 0.330722935464677*I],
   [ -0.376206602026623 + 

In [5]:
import shap
X = []
y1 = []
#y2 = []
for h in range(1):
    if h % 10==0:
        print(h/10)
    state = np.random.uniform(low=[-1.5,-5,-2,-3], high=[1.5,5,2,3], size=(4))

    X.append(state)
    eigenval, eigenvec = np.linalg.eig(np.array(JM.evalf(subs={'e1':state[0], 'e2':state[1], 'e3':state[2], 'e4':state[3]}), dtype=np.float64))
    #a1, a2 = np.real(eigenval[0]), np.imag(eigenval[0])
    #y1.append(a1)
    #y2.append(a2)
    y1.append(min(np.real(1)))
X = np.array(X)

0.0


TypeError: 'int' object is not iterable

In [6]:
np.real(eigenval)

array([  8.32662804,   0.1063693 , -23.02565834, -23.02565834])

In [7]:
eigenval

array([  8.32662804+0.j        ,   0.1063693 +0.j        ,
       -23.02565834+2.67529451j, -23.02565834-2.67529451j])

In [8]:
X

[array([ 1.10559851, -1.56989816,  0.78874222, -1.89187979])]

In [35]:
a,b,c = sympy.symbols('a b c')
m = sympy.Matrix([[-3,1,0], [-3,0,1], [-1+a,0,0]])

In [36]:
m

Matrix([
[   -3, 1, 0],
[   -3, 0, 1],
[a - 1, 0, 0]])

In [37]:
m.eigenvals()

{a**(1/3) - 1: 1,
 a**(1/3)*(-1/2 + sqrt(3)*I/2) - 1: 1,
 a**(1/3)*(-1/2 - sqrt(3)*I/2) - 1: 1}

In [21]:
h1 = c/3 - (3*b + 9*c + (3 - c)**2 - 9)/(3*(-27*a/2 - 81*b/2 - 81*c/2 + (3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3)/2 + (-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)**0.5/2 + 27/2)**(1/3)) - (-27*a/2 - 81*b/2 - 81*c/2 + (3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3)/2 + (-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)**0.5/2 + 27/2)**(1/3)/3 - 1

In [22]:
h2 =c/3 - (3*b + 9*c + (3 - c)**2 - 9)/(3*(-1/2 + sympy.sqrt(3)*I/2)*(-27*a/2 - 81*b/2 - 81*c/2 + (3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3)/2 + sympy.sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/2 + 27/2)**(1/3)) - (-1/2 + sympy.sqrt(3)*I/2)*(-27*a/2 - 81*b/2 - 81*c/2 + (3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3)/2 + sympy.sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/2 + 27/2)**(1/3)/3 - 1

In [30]:
h3 =c/3 - (3*b + 9*c + (3 - c)**2 - 9)/(3*(-1/2 - sympy.sqrt(3)*I/2)*(-27*a/2 - 81*b/2 - 81*c/2 + (3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3)/2 + sympy.sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/2 + 27/2)**(1/3)) - (-1/2 - sympy.sqrt(3)*I/2)*(-27*a/2 - 81*b/2 - 81*c/2 + (3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3)/2 + sympy.sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/2 + 27/2)**(1/3)/3 - 1

In [29]:
h1

c/3 - 0.097064496081811*(3*b + 9*c + (3 - c)**2 - 9)/(-a/3 - b - c + 2*(3 - c)**3/81 - (27 - 9*c)*(-b - 3*c + 3)/81 + (-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)**0.5/81 + 0.333333333333333)**0.333333333333333 - 1.14471424255333*(-a/3 - b - c + 2*(3 - c)**3/81 - (27 - 9*c)*(-b - 3*c + 3)/81 + (-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)**0.5/81 + 0.333333333333333)**0.333333333333333 - 1

In [24]:
h2

c/3 - 0.291193488245433*(3*b + 9*c + (3 - c)**2 - 9)/((-1.5 + 0.009*sqrt(3))*(-a/3 - b - c + 2*(3 - c)**3/81 - (27 - 9*c)*(-b - 3*c + 3)/81 + sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/81 + 0.333333333333333)**0.333333333333333) - 1.14471424255333*(-0.5 + 0.003*sqrt(3))*(-a/3 - b - c + 2*(3 - c)**3/81 - (27 - 9*c)*(-b - 3*c + 3)/81 + sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/81 + 0.333333333333333)**0.333333333333333 - 1

In [26]:
h3 = c/3 - (3*b + 9*c + (3 - c)**2 - 9)/(3*(-1/2 - sqrt(3)*I/2)*(-27*a/2 - 81*b/2 - 81*c/2 + (3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3)/2 + sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/2 + 27/2)**(1/3)) - (-1/2 - sqrt(3)*I/2)*(-27*a/2 - 81*b/2 - 81*c/2 + (3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3)/2 + sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/2 + 27/2)**(1/3)/3 - 1

-0.5658142006234398

In [31]:
h3

c/3 - 0.291193488245433*(3*b + 9*c + (3 - c)**2 - 9)/((-1.5 - 0.009*sqrt(3))*(-a/3 - b - c + 2*(3 - c)**3/81 - (27 - 9*c)*(-b - 3*c + 3)/81 + sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/81 + 0.333333333333333)**0.333333333333333) - 1.14471424255333*(-0.5 - 0.003*sqrt(3))*(-a/3 - b - c + 2*(3 - c)**3/81 - (27 - 9*c)*(-b - 3*c + 3)/81 + sqrt(-4*(3*b + 9*c + (3 - c)**2 - 9)**3 + (-27*a - 81*b - 81*c + 2*(3 - c)**3 - (27 - 9*c)*(-b - 3*c + 3) + 27)**2)/81 + 0.333333333333333)**0.333333333333333 - 1